In [ ]:
import numpy as np
from scipy.stats import norm, poisson

# Define the population distribution by age group
age = np.array([0.38, 0.33, 0.23, 0.06])
Total_population = 8700000  # Total population of the city

# Define the initial number of normal and covid beds
normal_bed = 650
covid_beds = 0
discount_factor=0.9
# Define the cost parameters
dos_normal = 500
dos_covid = 2000
alpha = 10000

# Define the range of beds to convert from normal to covid
beds_convert_range = np.arange(0, normal_bed + 1, 50)

# Define the transition function
def transitions(state, action):
    # Unpack the state
    age, Total_population, beds_normal, beds_covid = state
    
    # Calculate the number of requests for normal and covid beds
    requests_normal = np.sum(poisson.rvs(0.1, size=len(age)) * age)
    requests_covid = np.sum(poisson.rvs(0.05, size=len(age)) * age)
    
    # Calculate the number of discharges for normal and covid beds
    discharge_n = poisson.rvs(beds_normal)
    discharge_c = poisson.rvs(beds_covid,)
    
    # Calculate the number of denied requests for normal and covid beds
    denied_normal = max(requests_normal - beds_normal - action, 0)
    denied_covid = max(requests_covid - beds_covid, 0)
    
    # Calculate the new state after accounting for the action and the probabilistic transitions
    new_beds_normal = max(min(beds_normal + requests_normal - discharge_n - action, normal_bed), 0)
    new_beds_covid = max(min(beds_covid + requests_covid - discharge_c, normal_bed), 0)
    new_age = age + norm.rvs(scale=1, size=len(age))
    new_Total_population = max(min(Total_population + norm.rvs(scale=0.1), 10000000), 0)
    
    # Calculate the rewardss for the transition
    rewards = -dos_normal * denied_normal \
             -dos_covid * denied_covid \
             -alpha * action
    
    return ((new_age, new_Total_population, new_beds_normal, new_beds_covid), rewards)

# Define the value iteration function to solve the MDP
def value(theta=0.0001, discount_factor=0.9):
    V = np.zeros((age.size+2, normal_bed+1, 4))
    delta = float('inf')
    while delta > theta:
        delta = 0
        for age_group in range(age.size+2):
            for beds_normal in range(normal_bed+1):
                for week in range(4):
                    v = V[age_group, beds_normal, week]
                    action_values = []
                    for action in beds_convert_range:
                        new_state, rewards = transitions((age, Total_population, beds_normal, covid_beds), action)
                        next_age_group = np.clip(np.sum(new_state[0] * age / new_state[1]), 0,len(age)+1)
                        next_beds_normal = new_state[2]
                        next_week = week + 1 if week < 3 else 0
                        t= rewards + discount_factor 
                        action_values.append(t)
                    V[age_group, beds_normal, week] = max(action_values)
                    delta = max(delta, abs(v - V[age_group, beds_normal, week]))
    return V

# Solve the MDP
V = value()

# Extract the optimal policy
policy = np.zeros((age.size+2, normal_bed+1, 4))
for age_group in range(age.size+2):
    for beds_normal in range(normal_bed+1):
        for week in range(4):
            action_values = []
            for action in beds_convert_range:
                new_state, rewards = transitions((age, Total_population, beds_normal, covid_beds), action)
                next_age_group = np.clip(np.sum(new_state[0] * age / new_state[1]), 0, len(age)+1)
                next_beds_normal = new_state[2]
                next_week = week + 1 if week < 3 else 0
                action_values.append(rewards + discount_factor * V[next_age_group, next_beds_normal, next_week])
            policy[age_group, beds_normal, week] = beds_convert_range[np.argmax(action_values)]
# Print the optimal policy for each week
for week in range(4):
    print(f"Week {week+1}:")
    print("Age Group\tNormal Beds\tConvert to Covid Beds")
    for age_group in range(age.size+2):
        for beds_normal in range(normal_bed+1):
            print(f"{age_group}\t\t{beds_normal}\t\t{policy[age_group, beds_normal, week]}")

